In [48]:
import re
import sys
sys.path.insert(0, '..')
sys.path.insert(0, '../map-red')

import copy
import codecs
import itertools
from operator import itemgetter

import utils
import s9_archive
# from BlackSearch import BlackSearch, BooleanSearch

# import numpy as np
import pickle


In [52]:
# python TestMarks.py -s 9 -e -l 

all_index = False
if  all_index == True:
    ndx_name = '../all_index/povarenok_all_index.txt'
    bin_name = '../all_index/povarenok_all_s_backward.bin'
    len_name = '../all_index/povarenok_all_dlens.txt'

    rnk_name = '../all_index/povarenok_all_ranked.txt'
    url_name = 'C:\\data\\povarenok.ru\\all\\urls.txt'

else:
    ndx_name = '../map-red/data/povarenok1000_index.txt'
    bin_name = '../map-red/data/povarenok1000s_backward.bin'
    len_name = '../map-red/data/povarenok1000_dlens.txt'
    rnk_name = '../ranked.txt'
    

In [53]:
archiver =  s9_archive.Simple9Archiver()

In [54]:
bs = BooleanSearch(ndx_name, bin_name, archiver)     
br = BlackSearch(bs, lex=utils.MyLex(), dlen_name=len_name)

ValueError: Expecting object: line 1 column 2 (char 1)

In [22]:
class TextSearch(object):
    """ Пассажный алгоритм:
    
        Пассаж - фрагмент документа, размера, не превышающего заданный, в котором 
        встречаются все термы запроса, либо значительная часть термов запроса, 
        суммарный IDF которых превышает заданное ограничение. Набор нескольких 
        релевантных вхождений, которые расположены не слишком далеко друг от друга.

        Скользящее окно — это массив, каждый элемент которого соответствует слову из запроса.
        Двигаясь по релевантным вхождениям мы пытаемся связать их с ячейками скользящего окна.
        Каждый матчинг приводит к формированию пассажей.  
        
        Пассажный ранг документа - ранг лучшего пассажа в документе.
        Итоговый ранг: вес(tf-idf) + вес(пассажного алгоритма)   (+ вес(парного tf-idf))
    """
    def __init__(self, br,
                 stops_filename='./data/StopWords.txt',
                 use_synonyms=False,
                 synonyms_filename='./data/RusSyn.txt'):
        """ Constructor """
        self.min_len_meanful_word = 3
        with codecs.open(stops_filename, 'r', encoding='utf-8') as f_stops:
            self.stops = set(map(lambda line: line.strip().rstrip('\n'), f_stops.readlines() ))

        self.use_synonyms = use_synonyms
        if  use_synonyms:
            self.syns = {}
            with codecs.open(synonyms_filename, 'r', encoding='utf-8') as f_syn:
                for line in f_syn:
                    splt = line.rstrip().split(u'|')
                    if splt > 1:
                        norm = br.lex.norm(splt[0])
                        self.syns[norm] = list(set([norm] + splt[1].split(u',')))
        self.br = br
        self.params = [1.] * 7
                      # [6.029999999999999, 2.0699999999999985, 9.0, 1.08, \
                      #  2.0700000000000003, 3.059999999999999, 3.059999999999999]

    def filter_by_stops(self, query_norms):
        """ Return two (meanful and another) lists of words """
        means, stops = [], []
        for norm in query_norms:
            if (len(norm) >= self.min_len_meanful_word) \
                and (norm not in self.stops):
                means.append(norm)
            else:
                stops.append(norm)
        return  (means, stops)

    def formulate_request(self, query):
        """ Bool Search string
        
            Returns uniq_query_norms, query_norms_hashes, request
        """
        # Ordered query
        query__n_h = [ self.br.lex.normalize(word) for word in query.lower().split() ]
        # unique non-ordered normal forms of word in query
        query_norms = list(set(map(itemgetter(0),query__n_h)))

        query_means, query_stops = self.filter_by_stops(query_norms)
        
        query_syns = []
        if  self.use_synonyms:
            query_syns
            for i in xrange(len(query_means)):
                norm = query_means[i]
                try:
                    # Append synonims to request
                    query_syns += self.syns[norm]
                except: continue

        request = query_means + query_stops + query_syns
        return (query__n_h, query_means, request)

    def sliding_window(self, query__n_h, doc_text):
        """ Скользяцее окно """
        passages = []
        N = len(query__n_h)

        passage = []
        norms_psg = set()
        for norm, posit, hash in doc_text:

            if  norm in norms_psg:
                for i in xrange(len(passage)):
                    if passage[i][0] == norm:
                        del passage[i]
                        break
            else:
                norms_psg.add(norm)

            if len(passage) == N:
                in_psg.remove(passage[0][0])
                del passage[0]
            
            passage.append( (norm, posit, hash) )
            passages.append( copy.copy(passage) )
        return passages

    def passage_rank(self, passage, passage_norms, query_norms, query__n_h, doc_id):
        """ Каждый пассаж численно оценивается по следующим показателям:
            •	Полнота:      %слов из запроса в пассажей, все ли слова представлены
            •	Расстояние    от начала документа
            •	Кучность:     как число слов не из пассажа между словами пассажа
            •	Слово-форма:  различие слово-форм в пассаже
            •	Порядок слов: транспозиции
            •	tf-idf        ранг пассажа
            Returns the vector of passage characteristics
            -->  maximaze to get the best
        """
        # passage item: norm, posit, hash
        fullness = float(len(passage_norms)) / len(query_norms)

        psg_range = (passage[-1][1] - passage[0][1]) + 1.
        psg_range = float(len(passage) if psg_range < len(passage) else psg_range)
        compactness = 1. - (psg_range - len(passage)) / psg_range

        len_text = self.br.doc_lens[doc_id]
        close2start = 1. - float(passage[0][1]) / len_text

        words_form = 0.
        forms = [ (n,h) for n,p,h in passage ]
        for norm,hash in query__n_h:
            if (norm,hash) in forms:
                forms.remove( (norm,hash) )
                words_form += 1.
        words_form /= len(query__n_h)

        all_trs, eq_trs = 0., 0.
        q_transpositions = itertools.combinations(range(len(query__n_h)), 2)
        p_transpositions = itertools.combinations(range(len(passage)), 2)
        for i,j in q_transpositions:
            for p,q in p_transpositions:
                if  query__n_h[i][0] == passage[p][0] and \
                    query__n_h[j][0] == passage[q][0]:
                    eq_trs += 1.
                all_trs += 1.
        words_order = eq_trs / all_trs if all_trs else 0.
        # --> max
        return (fullness, compactness, close2start, words_form, words_order)

    def ranking(self, query_norms, query__n_h, query_index, doc_id, tfs, idfs):
        """ """
        doc_text = []
        for word, index in query_index.items():
            try:
                doc_index = index["ids"].index(doc_id)
            except: continue
            posits, hashes = self.br.bs.decode_posits_and_hashes_for_doc(index, doc_index)
            doc_text += zip([word] * min(len(posits), len(hashes)), posits, hashes)

        doc_text.sort(key=itemgetter(1))

        doc_passages = self.sliding_window(query__n_h, doc_text)

        doc_passages_ranges = []
        for i, passage in enumerate(doc_passages):
            passage_norms = list(set([ n for n,p,h in passage ]))

            doc_score = self.br.ranking(passage_norms, query_index, [doc_id], tfs, idfs)
            doc, tf_idf = doc_score[0]

            vector = list(self.passage_rank(passage, passage_norms, query_norms, query__n_h, doc_id))
            # НОРМИРОВАННЫЕ ВЕЛИЧИНЫ:
            #   1. Полнота
            #   2. Компактность
            #   3. Близость к началу
            #   4. Слово-форма
            #   5. Порядок слов
            #   6. tf-idf пассажа
            # Итоговый ранк пассажа, как линейная комбинация
            # параметров пассажа с коэффициентами = 1.
            # --->> MAXIMIZE
            doc_passages_ranges.append( vector + [tf_idf] )
            
        doc_passages_ranges.sort(key=lambda x: sum(x), reverse=True)
        return  doc_passages_ranges[0]
    

In [5]:
def search(self, query, max_n_docs=1000, use_synonyms=False, mark_id=None):
    """ Returns the  doc_scores: (doc_id, total_rank) """
    query__n_h, query_means, request = self.formulate_request(query)

    if 1:
        inv_q = self.br.lex.incorrect_keyboard_layout(query)
        if  inv_q:
            inv_q__n_h, inv_q_means, inv_q_req = self.formulate_request(inv_q)
            # Append inversed to request line
            query__n_h += inv_q__n_h
            query_means += inv_q_means
            request += inv_q_req

        trs_q = self.br.lex.transliterate(query)
        if  trs_q:
            trs_q__n_h, trs_q_means, trs_q_req = self.formulate_request(trs_q)
            # Append transliterated to request line
            query__n_h += trs_q__n_h
            query_means += trs_q_means
            request += trs_q_req

        query_index, indexed_doc_ids = self.br.search(request,
                                                      up=["ids", "lens", "posits", "hashes"],
                                                      verbose=True)

        # query_index  is  { norm: [ "ids", "lens", "posits", "hashes" ], ... }
        query_norms = query_index.keys()
        query__n_h = [ (norm, hash)  for norm, hash in query__n_h  if norm in query_norms ]

        # If there are STOPs - OK, if not, it is also OK!
        intersect_doc_ids = set(indexed_doc_ids)
        for word, index in query_index.items():
            if word in query_means:
                intersect_doc_ids &= set(index['ids'])
        # ?? synonyms
        intersect_doc_ids = list(intersect_doc_ids)

        tfs, idfs  = self.br.tf_idf(query_index, intersect_doc_ids)
        doc_scores = self.br.ranking(query_norms, query_index, intersect_doc_ids, tfs, idfs)

        # choose documents to ranking by passage algorithm
        doc_bm25_scores = doc_scores[:max_n_docs]

        doc_pssg_scores = []
        for doc_id, score in doc_bm25_scores:
            best_passage_rank = self.ranking(query_norms, query__n_h,
                                             query_index, doc_id,
                                             tfs, idfs)
            doc_pssg_scores.append( (doc_id, best_passage_rank) )

        doc_pssg_scores.sort(key=itemgetter(0))
        doc_bm25_scores.sort(key=itemgetter(0))

        # total rank
        doc_scores = map(lambda x, y: (x[0], sum(x[1]) + y[1]), doc_pssg_scores, doc_bm25_scores)
        doc_scores.sort(key=itemgetter(1), reverse=True)

        doc_ids =  [ doc_id for doc_id, score in doc_scores ]

        # if mark_id:
        #     try:
        #         real_no = doc_ids.index(mark_id)
        #     except: return doc_ids
        # 
        #     print '\nreal_no', real_no
        #     vectors = np.ndarray(doc_pssg_scores)
        #     print vectors.shape
        # 
        #     vs = np.ndarray(doc_bm25_scores)[:,1]
        #     print vs.shape
        #     vectors = np.hstack([vectors,  vs])
        # 
        #     with open(mark_id + 'ranged.txt', 'w') as f:
        #         pickle.dump(vectors, f)

        # if 0:
        #     with open(mark_id + 'ranged.txt', 'r') as f:
        #         vectors = pickle.loads(f.read())
        # 
        # 
        #     # if real_no >= 3:
        # 
        #     # vectors = np.array(np.array([d[1] for d in doc_pssg_scores]))
        #     # vectors = np.hstack([vectors, np.array([d[1] for d in doc_bm25_scores]) ])
        #     
        #     # fullness, compactness, close2start, words_form, words_order, bm25
        #     # stats, means = zip(*( (np.partition(vals, k)[k], np.mean(vals)) for vals in vectors[:,1:].T ))
        #     
        #     means = [ np.mean(vals) for vals in vectors[:,1:].T ]
        #     vars  = [  np.var(vals) for vals in vectors[:,1:].T ]
        # 
        #     params = np.array([1.] * 7);
        #     for i, coord in enumerate(vectors[real_no,:]):
        #         if   means[i] <  coord:
        #             params[i] *= vars[i]
        #         elif means[i] >  coord:
        #             params[i] *= 1. / vars[i]
        #      
        #     # sttl = doc_pssg_scores[real_no][1] + [ doc_bm25_scores[real_no][1] ]
        #     # print sttl
        #     # params = [0.01] * 7;
        #     # j = np.argmax(sttl)
        #     # for k,s in enumerate(sttl):
        #     #     if  s > 0.9 or k == j:
        #     #         params[k] = 1.
        # 
        #     print j, params
        #     self.params = np.array(self.params) + params
        # 
        #         # k  = -5
        #         # vectors = np.array(doc_pssg_scores)
        #         # vectors = np.hstack([vectors, np.array(doc_bm25_scores_)[:,1] ])
        #         # # fullness, compactness, close2start, words_form, words_order, bm25
        #         # stats, means = zip(*( (np.partition(vals, k)[k], np.mean(vals)) for vals in vectors[:,1:].T ))
        #         # 
        #         # for i, coord in enumerate(vectors[real_no,:]):
        #         #     if  stats[i] <= coord:
        #         #         params[i] += means[i]
        #         #     else:
        #         #         params[i] += 1. / means[i]
        #         # 
        #         # with open('params.txt', 'a') as f:
        #         #     print >>f, params
        #         # self.params += params
        #     # else:
        #         # self.parmas += [1.] * len(self.params)
        #         # with open('params.txt', 'a') as f:
        #         #     print >>f, [1.] * len(self.params)
        # # except: pass
        #     # self.parmas += [1.] * len(self.params)
        #     # with open('params.txt', 'a') as f:
        #     #     print >>f, '---';

        return doc_ids

In [46]:
ts = TextSearch(br)

query   = u'пирог с камбалой'  #  u'рецепт блюда из птицы'  #  u'картинки эпел джек'
mark_id = 81390                #  193112                    #  193860

self = ts

In [24]:
def prnt(x):
    print u' '.join(x)

In [25]:
query__n_h, query_means, request = self.formulate_request(query)

inv_q = self.br.lex.incorrect_keyboard_layout(query)
if  inv_q:
    inv_q__n_h, inv_q_means, inv_q_req = self.formulate_request(inv_q)
    # Append inversed to request line
    query__n_h += inv_q__n_h
    query_means += inv_q_means
    request += inv_q_req

trs_q = self.br.lex.transliterate(query)
if  trs_q:
    trs_q__n_h, trs_q_means, trs_q_req = self.formulate_request(trs_q)
    # Append transliterated to request line
    query__n_h += trs_q__n_h
    query_means += trs_q_means
    request += trs_q_req
    
prnt(request)

камбала пирог с


In [28]:
query_index, indexed_doc_ids = self.br.search(request,
                                              up=["ids", "lens", "posits", "hashes"],
                                              verbose=True)

# query_index  is  { norm: [ "ids", "lens", "posits", "hashes" ], ... }
query_norms = query_index.keys()
query__n_h = [ (norm, hash)  for norm, hash in query__n_h  if norm in query_norms ]


In [32]:

for k,v in query_index.items():
    prnt(k)
    print v['ids'][-1], len(v['ids'])

с
199456 103899
п и р о г
199456 181331
к а м б а л а
197436 434


In [29]:
# If there are STOPs - OK, if not, it is also OK!
intersect_doc_ids = set(indexed_doc_ids)
for word, index in query_index.items():
    if word in query_means:
        intersect_doc_ids &= set(index['ids'])
# ?? synonyms
# print intersect_doc_ids


set([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199])
[18434, 143362, 14355, 61469, 131102, 12324, 26665, 192554, 137287, 24650, 24651, 51279, 122960, 116820,

In [38]:
tfs, idfs  = self.br.tf_idf(query_index, intersect_doc_ids)
doc_scores = self.br.ranking(query_norms, query_index, intersect_doc_ids, tfs, idfs)
# choose documents to ranking by passage algorithm
doc_bm25_scores = doc_scores[:1000]
# print doc_bm25_scores

In [47]:
doc_pssg_scores = []
for doc_id, score in doc_bm25_scores:
    best_passage_rank = self.ranking(query_norms, query__n_h,
                                     query_index, doc_id,
                                     tfs, idfs)
    doc_pssg_scores.append( (doc_id, best_passage_rank) )

doc_pssg_scores.sort(key=itemgetter(0))
doc_bm25_scores.sort(key=itemgetter(0))

430


IndexError: list index out of range

In [36]:
# total rank
doc_scores = map(lambda x, y: (x[0], sum(x[1]) + y[1]), doc_pssg_scores, doc_bm25_scores)
doc_scores.sort(key=itemgetter(1), reverse=True)

doc_ids =  [ doc_id for doc_id, score in doc_scores ]
print doc_ids


[148581, 144387, 99450, 130918, 68408, 144917, 168083, 103889, 98998, 134112]


In [ ]:
if mark_id in doc_ids:
    print "OK"
    print doc_ids.index(mark_id)

In [ ]:
def brute_coefs(TextSearch, marks):
    """ Алгоритм  дифференциaльной эволюции (differential evolution) 

        На каждой итерации генерируется множество векторов, называемых поколением, 
        случайным образом комбинируя векторы из предыдущего поколения.

        Для каждого вектора x_i из старого поколения выбираются три различных случайных вектора
        v_1, v_2, v_3 среди векторов старого поколения, за исключением самого вектора x_i, и
        генерируется так называемый мутантный вектор (mutant vector) по формуле:

        v = v_1 + F \cdot (v_2 - v_3),

        где F — один из параметров метода, некоторая положительная действительная константа в интервале [0, 2].

        Над мутантным вектором v выполняется операция «скрещивания» (crossover), состоящая в том, что некоторые
        его координаты замещаются соответствующими координатами из исходного вектора x_i (каждая координата
        замещается с некоторой вероятностью, которая также является еще одним из параметров этого метода).
        Полученный после скрещивания вектор называется пробным вектором (trial vector). 
        Если он оказывается лучше вектора x_i (то есть значение целевой функции стало меньше),
        то в новом поколении вектор x_i заменяется на пробный вектор, а в противном случае — остаётся x_i.
    """

    # fit 
    self.params
    return

In [ ]:
def pair_tf_idf(TextSearch):
    """ Вхождение терма в документ учитывается только в 
        том случае, если оно находится в документе на 
        расстоянии, не превышающим заданное от хотя бы 
        одного из стоящих рядом с ним термов запроса.
    """
    # TODO
    return